# Machine Learning parte 2: Otimização com exploração aleatória

* Este estudo é uma continuação do estudo de [ML parte 1: Otimização de modelos através de hiperparâmetros](https://colab.research.google.com/drive/1zkmLyzgb1jqOybg_ZRvQR9_uk0_AIYHr?usp=sharing).

## Abertura do Dataset

In [1]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [2]:
# simulação de uma péssima organização dos dados
dados_azar = dados.sort_values('vendido', ascending = True)
x_azar = dados_azar[['preco', 'idade_do_modelo', 'km_por_ano']]
y_azar = dados_azar['vendido']

# Busca aleatória (Randomized Search CV)

* Define-se um espaço de parâmetros e uma quantidade de iterações que o algoritmo deve executar, menor do que o espaço completo.

* A exploração é feita a quantidade de vezes definida, portanto nem todo o espaço será explorado, reduzindo o custo do processo.

* Como definido no código abaixo, o espaço de busca resulta em, no máximo, 32 variações de modelo. Como a escolha será aleatória, nem todos serão explorados.

In [3]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold

SEED = 301
np.random.seed(SEED)

espaco_de_parametros = { 
    "max_depth" : [3, 5],
    "min_samples_split": [32, 64, 128],
    "min_samples_leaf": [32, 64, 128],
    "criterion": ["gini", "entropy"]
}

busca = RandomizedSearchCV(DecisionTreeClassifier(),
                    espaco_de_parametros,
                    cv = KFold(n_splits = 5, shuffle=True),
                    random_state = SEED,
                    n_iter = 16) # apenas 16 variações serão exploradas, ao invés de 36

busca.fit(x_azar, y_azar)
resultados = pd.DataFrame(busca.cv_results_)

* O treino foi feito com cross-validation, então a validação será da mesma forma.

In [4]:
def imprime_score(scores):
  media = scores.mean()
  desvio_padrao = scores.std()

  print(f'Accuracy mean = {media*100:.2f}%')
  print(f'Accuracy interval = [{(media - 2*desvio_padrao)*100 :.2f}%, {(media + 2*desvio_padrao)*100 :.2f}%]')

In [5]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(busca, x_azar, y_azar, cv = KFold(n_splits=5, shuffle=True))

In [6]:
imprime_score(scores)

Accuracy mean = 78.71%
Accuracy interval = [77.49%, 79.93%]


In [7]:
melhor = busca.best_estimator_
print(melhor)

DecisionTreeClassifier(max_depth=3, min_samples_leaf=32, min_samples_split=64)


* O estimador encontrado pela busca aleatória foi semelhante ao encontrado no estudo anterior, com uma diferença no min_samples_split.

* O tempo gasto para encontrar esta solução foi muito menor, apesar a diferença pequena.

## Espaço de parâmetros mais abrangente

In [16]:
from scipy.stats import randint

SEED = 301
np.random.seed(SEED)

espaco_de_parametros = { 
    "max_depth" : [3, 5, 10, 15, 20, 30, None],
    "min_samples_split": randint(32, 129), # vai gerando números aleatórios à medida que são requisitados
    "min_samples_leaf": randint(32, 129),  # não apenas uma única vez
    "criterion": ["gini", "entropy"]
}

busca = RandomizedSearchCV(DecisionTreeClassifier(),
                    espaco_de_parametros,
                    cv = KFold(n_splits = 5, shuffle=True),
                    random_state = SEED,
                    n_iter = 16) # apenas 16 variações serão exploradas, ao invés de 36

busca.fit(x_azar, y_azar)
resultados = pd.DataFrame(busca.cv_results_)

In [17]:
scores = cross_val_score(busca, x_azar, y_azar, cv = KFold(n_splits=5, shuffle=True))
imprime_score(scores)

Accuracy mean = 78.71%
Accuracy interval = [77.49%, 79.93%]


In [10]:
melhor = busca.best_estimator_
print(melhor)

DecisionTreeClassifier(criterion='entropy', max_depth=3, min_samples_leaf=71,
                       min_samples_split=100)


* A acurácia média foi um pouco maior e o intervalo mais afunilado, em comparação com o estimador anterior.

* Somente um hiperparâmetro permaneceu, a profundidade máxima 3.

## Análise por Score médio

In [24]:
resultados_ordenados_pela_media = resultados.sort_values('mean_test_score', ascending = False)

for indice, linha in resultados_ordenados_pela_media.iterrows():
  mts = linha['mean_test_score']
  sts = linha['std_test_score']
  params = linha['params']
  print(f'{mts:.4f} \t +/- {sts*2:.4f} \t {params}')

0.7870 	 +/- 0.0192 	 {'criterion': 'entropy', 'max_depth': 3, 'min_samples_leaf': 71, 'min_samples_split': 100}
0.7839 	 +/- 0.0237 	 {'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 73, 'min_samples_split': 72}
0.7839 	 +/- 0.0237 	 {'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 64, 'min_samples_split': 67}
0.7810 	 +/- 0.0167 	 {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 108, 'min_samples_split': 110}
0.7804 	 +/- 0.0194 	 {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 125, 'min_samples_split': 59}
0.7799 	 +/- 0.0124 	 {'criterion': 'gini', 'max_depth': 15, 'min_samples_leaf': 103, 'min_samples_split': 96}
0.7794 	 +/- 0.0208 	 {'criterion': 'gini', 'max_depth': 15, 'min_samples_leaf': 126, 'min_samples_split': 84}
0.7793 	 +/- 0.0204 	 {'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 124, 'min_samples_split': 88}
0.7792 	 +/- 0.0093 	 {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 101, 'min_samples_split': 5

## Aumento na quantidade de iterações

In [25]:
SEED = 301
np.random.seed(SEED)

espaco_de_parametros = { 
    "max_depth" : [3, 5, 10, 15, 20, 30, None],
    "min_samples_split": randint(32, 129), 
    "min_samples_leaf": randint(32, 129), 
    "criterion": ["gini", "entropy"]
}

busca = RandomizedSearchCV(DecisionTreeClassifier(),
                    espaco_de_parametros,
                    cv = KFold(n_splits = 5, shuffle=True),
                    random_state = SEED,
                    n_iter = 32)

busca.fit(x_azar, y_azar)
resultados = pd.DataFrame(busca.cv_results_)

In [34]:
resultados_ordenados_pela_media = resultados.sort_values('mean_test_score', ascending = False)

for indice, linha in resultados_ordenados_pela_media[:5].iterrows():
  mts = linha['mean_test_score']
  sts = linha['std_test_score']
  params = linha['params']
  print(f'{mts:.4f} \t +/- {sts*2:.4f} \t {params}')

0.7801 	 +/- 0.0203 	 {'bootstrap': False, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 32, 'min_samples_split': 64, 'n_estimators': 10}
0.7784 	 +/- 0.0202 	 {'bootstrap': True, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 32, 'min_samples_split': 128, 'n_estimators': 10}
0.7782 	 +/- 0.0300 	 {'bootstrap': False, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 64, 'min_samples_split': 64, 'n_estimators': 10}
0.7779 	 +/- 0.0271 	 {'bootstrap': False, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 64, 'min_samples_split': 64, 'n_estimators': 10}
0.7776 	 +/- 0.0332 	 {'bootstrap': False, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 64, 'min_samples_split': 64, 'n_estimators': 100}


* Apesar do aumento na quantidade de iterações, o melhor resultado encontrado foi exatamente o mesmo do anterior, com 16 iterações.

# Comparando GridSearchCV com o RandomizedSearch

* O GridSearch traz mais certeza da análise sobre um espaço de parâmetros, mas o RandomizedSearch traz muito mais controle sobre o custo computacional.

* A partir deste ponto, será usado o RandomTreeClassifier nos modelos.
  * https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

### Grid Search

In [29]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import time

SEED=301
np.random.seed(SEED)

espaco_de_parametros = { # 144 combinações de parâmetros
    "max_depth" : [3, 5],
    "min_samples_split" : [32, 64, 128],
    "min_samples_leaf" : [32, 64, 128],
    "criterion" : ["gini", "entropy"],
    "n_estimators" : [10, 100], # Quantas árvores serão criadas
    "bootstrap" : [True, False]
}

init_time = time.time()
busca = GridSearchCV(RandomForestClassifier(),
                    espaco_de_parametros,
                    cv = KFold(n_splits = 5, shuffle=True))

busca.fit(x_azar, y_azar)
end_time = time.time()
tempo = end_time - init_time

resultados = pd.DataFrame(busca.cv_results_)

In [36]:
print(f'Tempo decorrido: {tempo/60:.1f} minutos\n')

resultados_ordenados_pela_media = resultados.sort_values('mean_test_score', ascending = False)

for indice, linha in resultados_ordenados_pela_media[:5].iterrows():
  mts = linha['mean_test_score']
  sts = linha['std_test_score']
  params = linha['params']
  print(f'{mts:.4f} \t +/- {sts*2:.4f} \t {params}')

Tempo decorrido: 4.0 minutos

0.7801 	 +/- 0.0203 	 {'bootstrap': False, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 32, 'min_samples_split': 64, 'n_estimators': 10}
0.7784 	 +/- 0.0202 	 {'bootstrap': True, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 32, 'min_samples_split': 128, 'n_estimators': 10}
0.7782 	 +/- 0.0300 	 {'bootstrap': False, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 64, 'min_samples_split': 64, 'n_estimators': 10}
0.7779 	 +/- 0.0271 	 {'bootstrap': False, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 64, 'min_samples_split': 64, 'n_estimators': 10}
0.7776 	 +/- 0.0332 	 {'bootstrap': False, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 64, 'min_samples_split': 64, 'n_estimators': 100}


* O tempo de execução das duas células a seguir é longo o suficiente para ser abortado pelo Colab, mas este teste pode ser executado localmente.

In [38]:
#init_time = time.time()
#scores = cross_val_score(busca, x_azar, y_azar, cv = KFold(n_splits=5, shuffle=True))
#end_time = time.time()
#tempo = end_time - init_time

#print(f'Tempo decorrido: {tempo/60:.1f} minutos\n')
#imprime_score(scores)

In [43]:
melhor = busca.best_estimator_
print(melhor)

RandomForestClassifier(bootstrap=False, max_depth=5, min_samples_leaf=32,
                       min_samples_split=32)


### Random Forest

In [40]:
SEED=301
np.random.seed(SEED)

espaco_de_parametros = { # 144 combinações de parâmetros
    "max_depth" : [3, 5],
    "min_samples_split" : [32, 64, 128],
    "min_samples_leaf" : [32, 64, 128],
    "criterion" : ["gini", "entropy"],
    "n_estimators" : [10, 100], # Quantas árvores serão criadas
    "bootstrap" : [True, False]
}

init_time = time.time()
busca = RandomizedSearchCV(RandomForestClassifier(),
                    espaco_de_parametros,
                    cv = KFold(n_splits = 5, shuffle=True),
                    n_iter = 20)

busca.fit(x_azar, y_azar)
end_time = time.time()
tempo = end_time - init_time

resultados = pd.DataFrame(busca.cv_results_)

In [41]:
print(f'Tempo decorrido: {tempo/60:.1f} minutos\n')

Tempo decorrido: 0.6 minutos



* Tempo decorrido muito menor que com o GridSearch.

In [42]:
resultados_ordenados_pela_media = resultados.sort_values('mean_test_score', ascending = False)

for indice, linha in resultados_ordenados_pela_media[:5].iterrows():
  mts = linha['mean_test_score']
  sts = linha['std_test_score']
  params = linha['params']
  print(f'{mts:.4f} \t +/- {sts*2:.4f} \t {params}')

0.7761 	 +/- 0.0251 	 {'n_estimators': 100, 'min_samples_split': 32, 'min_samples_leaf': 32, 'max_depth': 5, 'criterion': 'gini', 'bootstrap': False}
0.7756 	 +/- 0.0227 	 {'n_estimators': 100, 'min_samples_split': 32, 'min_samples_leaf': 128, 'max_depth': 3, 'criterion': 'gini', 'bootstrap': False}
0.7756 	 +/- 0.0238 	 {'n_estimators': 100, 'min_samples_split': 64, 'min_samples_leaf': 32, 'max_depth': 5, 'criterion': 'entropy', 'bootstrap': True}
0.7755 	 +/- 0.0320 	 {'n_estimators': 10, 'min_samples_split': 32, 'min_samples_leaf': 64, 'max_depth': 3, 'criterion': 'entropy', 'bootstrap': False}
0.7754 	 +/- 0.0346 	 {'n_estimators': 10, 'min_samples_split': 32, 'min_samples_leaf': 32, 'max_depth': 5, 'criterion': 'gini', 'bootstrap': True}


In [44]:
melhor = busca.best_estimator_
print(melhor)

RandomForestClassifier(bootstrap=False, max_depth=5, min_samples_leaf=32,
                       min_samples_split=32)


In [45]:
init_time = time.time()
scores = cross_val_score(busca, x_azar, y_azar, cv = KFold(n_splits=5, shuffle=True))
end_time = time.time()
tempo = end_time - init_time

print(f'Tempo decorrido: {tempo/60:.1f} minutos\n')
imprime_score(scores)

Tempo decorrido: 2.8 minutos

Accuracy mean = 77.59%
Accuracy interval = [76.47%, 78.71%]


### Random Forest com espaço maior

In [47]:
SEED=301
np.random.seed(SEED)

espaco_de_parametros = { # mais de 10 milhões de combinações de parâmetros
    "max_depth" : randint(3, 6),
    "min_samples_split" : randint(32, 129),
    "min_samples_leaf" : randint(32, 129),
    "criterion" : ["gini", "entropy"],
    "n_estimators" : randint(10, 101),
    "bootstrap" : [True, False]
}

init_time = time.time()
busca = RandomizedSearchCV(RandomForestClassifier(),
                    espaco_de_parametros,
                    cv = KFold(n_splits = 5, shuffle=True),
                    n_iter = 80)

busca.fit(x_azar, y_azar)
end_time = time.time()
tempo = end_time - init_time

resultados = pd.DataFrame(busca.cv_results_)

In [48]:
print(f'Tempo decorrido: {tempo/60:.1f} minutos\n')

Tempo decorrido: 2.1 minutos



In [49]:
resultados_ordenados_pela_media = resultados.sort_values('mean_test_score', ascending = False)

for indice, linha in resultados_ordenados_pela_media[:5].iterrows():
  mts = linha['mean_test_score']
  sts = linha['std_test_score']
  params = linha['params']
  print(f'{mts:.4f} \t +/- {sts*2:.4f} \t {params}')

0.7790 	 +/- 0.0246 	 {'bootstrap': False, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 84, 'min_samples_split': 89, 'n_estimators': 48}
0.7785 	 +/- 0.0311 	 {'bootstrap': False, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 32, 'min_samples_split': 96, 'n_estimators': 18}
0.7779 	 +/- 0.0315 	 {'bootstrap': False, 'criterion': 'entropy', 'max_depth': 4, 'min_samples_leaf': 121, 'min_samples_split': 47, 'n_estimators': 27}
0.7775 	 +/- 0.0238 	 {'bootstrap': False, 'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 96, 'min_samples_split': 98, 'n_estimators': 11}
0.7771 	 +/- 0.0287 	 {'bootstrap': True, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 63, 'min_samples_split': 88, 'n_estimators': 69}


In [50]:
melhor = busca.best_estimator_
print(melhor)

RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=5,
                       min_samples_leaf=84, min_samples_split=89,
                       n_estimators=48)


In [51]:
init_time = time.time()
scores = cross_val_score(busca, x_azar, y_azar, cv = KFold(n_splits=5, shuffle=True))
end_time = time.time()
tempo = end_time - init_time

print(f'Tempo decorrido: {tempo/60:.1f} minutos\n')
imprime_score(scores)

Tempo decorrido: 9.2 minutos

Accuracy mean = 77.24%
Accuracy interval = [75.49%, 78.99%]


# Treino, teste, validação - otimização sem validação cruzada

* Por ser cara computacionalmente, a otimização cruzada pode se tornar inviável em alguns casos.

* Netse estudo, pode haver a disisão por 3 fases:
  * Fases de **treino** e **teste**, com cross validation
  * **Validação**, com nested validation (cross_val_score)

* Em vez de utilizar o KFold, pode ser usado o StratifiedShuffleSplit, que fará a divisão preservando a porcentagem que cada classe abrange nos dados dentro se cada subconjunto.
  * https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html

In [53]:
from sklearn.model_selection import train_test_split

SEED=301
np.random.seed(SEED)

# 60% treino
# 20% teste
# 20# validação

# separando 20% dos dados para validação e 80% que serão treino e teste
x_treino_teste, x_validacao, y_treino_teste, y_validacao = train_test_split(x_azar, y_azar, test_size = 0.2, shuffle = True, stratify = y_azar)

In [54]:
from sklearn.model_selection import StratifiedShuffleSplit

SEED=301
np.random.seed(SEED)

# 25% dos 80%, que é 20% do total, irá para o conjunto de teste
split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.25)

espaco_de_parametros = {
    "max_depth" : randint(3, 6),
    "min_samples_split" : randint(32, 129),
    "min_samples_leaf" : randint(32, 129),
    "criterion" : ["gini", "entropy"],
    "n_estimators" : randint(10, 101),
    "bootstrap" : [True, False]
}

init_time = time.time()
busca = RandomizedSearchCV(RandomForestClassifier(),
                    espaco_de_parametros,
                    cv = split,
                    n_iter = 5)

busca.fit(x_treino_teste, y_treino_teste)
end_time = time.time()
tempo = end_time - init_time

resultados = pd.DataFrame(busca.cv_results_)

* Neste caso, a validação deve ser feita com um conjunto de dados que nunca entrou em contato com o modelo, amostras novas. Por isso, a separação de um **conjunto de validação** deve ser feita antes.

In [58]:
init_time = time.time()
scores = cross_val_score(busca, x_validacao, y_validacao, cv = split)
end_time = time.time()
tempo = end_time - init_time

print(f'Tempo decorrido: {tempo:.6f} segundos\n')
scores

Tempo decorrido: 0.822937 segundos



array([0.76])

* Há apenas um valor de acurácia porque o teste é feito uma única vez (n_splits = 1).

* Aumentando-se o n_splits do StratifiedShuffleSplit o teste ficaria mais parecido com uma validação cruzada, mas as proporções das classes podem ser diferentes. Além disso, o cross_val_score voltaria a demorar mais.